In [1]:
import os
os.chdir('C:/Users/gudo/Desktop/cyclehunter/')
import cyclehunter
from cyclehunter import *
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import eig
import matplotlib.pyplot as plt
import time

# PhiK Class

Most relevant methods are `generate_states`, `prime_orbits` and `hunt`.
(I need to change from orbits to cycles oops..)

Class instantiation expects $n$, $k$, and $\mu^2$.

In [2]:
n = 3
k = 3
musqr = 5
x = PhiK(n, k, musqr)


Relevant attributes are `n`, `k`, `musqr`, `states`. The `states` are empty until we populate them via `generate_states`

In [3]:
print(type(x.states))

<class 'NoneType'>


In [4]:
x = x.generate_states(prime=False)

In [5]:
print(type(x.states))

<class 'numpy.ndarray'>


Prime orbits are default, but can be set to False and then reproduced by calling `prime_orbits` function

In [6]:
x.states = x.prime_orbits()

In [7]:
x.states

array([[-1, -1,  0],
       [-1, -1,  1],
       [ 0, -1,  0],
       [ 0, -1,  1],
       [ 1, -1,  0],
       [ 1, -1,  1],
       [ 0,  0,  1],
       [ 1,  0,  1]])

The numerical optimization routine is built into the class as opposed to be a separate module like it is in `orbithunter`;
this package's sister package.

In [8]:
converged_states = x.hunt()

Before after comparison

In [9]:
x.states 

array([[-1, -1,  0],
       [-1, -1,  1],
       [ 0, -1,  0],
       [ 0, -1,  1],
       [ 1, -1,  0],
       [ 1, -1,  1],
       [ 0,  0,  1],
       [ 1,  0,  1]])

In [10]:
converged_states

array([[-1.06677926, -1.06677926, -0.33060463],
       [-1.19266331, -1.19266331,  1.32651313],
       [-0.19883715, -1.1537165 , -0.19883715],
       [ 0.        , -1.26491094,  1.26491094],
       [ 1.26491094, -1.26491094,  0.        ],
       [ 1.19266331, -1.32651313,  1.19266331],
       [ 0.19883715,  0.19883715,  1.1537165 ],
       [ 1.06677926,  0.33060463,  1.06677926]])

Technically, I should be returning the optimization result to ensure convergence, however the two functions for the
equations of motion/cost function and the gradient of the cost function are: `eqn`, `cost`, `costgrad`.

In [11]:
x.states = converged_states

Cost is the $L_2$ norm squared of the governing equations `1/2 F^2`

In [12]:
x.cost()

1.1815044184388797e-10

Because it is the squared loss, we can expect the tolerance of each individual site in

In [13]:
x.eqn()

array([[-1.03063594e-06, -1.03063594e-06,  5.52687077e-07],
       [ 1.45450969e-06,  1.45450969e-06,  1.00265457e-05],
       [-7.90793613e-08, -1.62385988e-06, -7.90793613e-08],
       [ 0.00000000e+00, -2.03817493e-06,  2.03817493e-06],
       [ 2.03817493e-06, -2.03817493e-06,  0.00000000e+00],
       [-1.45450969e-06, -1.00265457e-05, -1.45450969e-06],
       [ 7.90793613e-08,  7.90793613e-08,  1.62385988e-06],
       [ 1.03063594e-06, -5.52687077e-07,  1.03063594e-06]])

Cost functional `1/2 F^2` is a scalar valued function; therefore the gradient is a vector, not a matrix.

In [14]:
x.costgrad()

array([ 1.08567307e-05,  1.08567307e-05,  5.02395572e-06, -3.23339051e-05,
       -3.23339051e-05, -1.97369425e-04,  1.19628119e-06,  2.12130064e-05,
        1.19628119e-06,  0.00000000e+00,  3.26107890e-05, -3.26107890e-05,
       -3.26107890e-05,  3.26107890e-05,  0.00000000e+00,  3.23339051e-05,
        1.97369425e-04,  3.23339051e-05, -1.19628119e-06, -1.19628119e-06,
       -2.12130064e-05, -1.08567307e-05, -5.02395572e-06, -1.08567307e-05])

In [ ]:
def rotate(A):
    x = A[len(A) - 1]
    for i in range(len(A) - 1, 0, -1):
        A[i] = A[i - 1];
    A[0] = x;
    return A


def checkCyclic(A, B):
    """ Checks if two orbits are members of the same group orbit
    
    A: 
    
    """
    return ', '.join(map(str, A)) in ', '.join(map(str, B)) 

def primeOrbits(initial_conditions, check_neg=False, check_rev=False): 
    #initial conditions should be you entire list of possible shadow state configurations
    #check_neg is a value that takes either 1 or 0 where if it is 1, it will check for phi to negative phi symmetry
    initial_conditions[initial_conditions == 1] = 3
    initial_conditions[initial_conditions == 0] = 2
    initial_conditions[initial_conditions == -1] = 1
    # here i am just changing my shadow state values to a different symbolic alphabet that will work better
    double_cycles = np.append(initial_conditions,initial_conditions,axis=1)
    # double_cycles is each shadow state repeated so that it is twice its length. This is used show checking for cyclic
    # permutations as every permunation exists in the orbit as if it goes through it twice. Ex: all cyclic permutation of 123 
    # exist somwhere in 123123
    i = 0
    while i < np.shape(initial_conditions)[0]: #looping through each row of the initial conditions
        j = np.shape(initial_conditions)[0] - 1
        while j > i: #looping rows of double_cycles, starting at the bottomw and ending before the row of the current 
                     #orbit we are checking
            if checkCyclic(initial_conditions[i],double_cycles[j]) == True:
                initial_conditions = np.delete(initial_conditions, j, 0) 
                double_cycles = np.delete(double_cycles, j, 0)    # if a orbit string exists in the double_cycle of of another
            j = j - 1                                             # orbit, delete one of the orbits
        i = i + 1
    if check_neg == 1:
        initial_conditions[initial_conditions == 1] = -1 #if we want to check if cycles are just negatives of another cycle
        initial_conditions[initial_conditions == 2] = 0
        initial_conditions[initial_conditions == 3] = 1
        initial_conditions = initial_conditions*(-1)     # have to first convert to shadow states in order to apply negative
        initial_conditions[initial_conditions == 1] = 3  # sign to states, then convert back the the 1 2 3 alphabet
        initial_conditions[initial_conditions == 0] = 2
        initial_conditions[initial_conditions == -1] = 1
        i = 0
        while i < np.shape(initial_conditions)[0]:
            j = np.shape(initial_conditions)[0] - 1
            while j > i:
                if checkCyclic(initial_conditions[i],double_cycles[j]) == True:
                    initial_conditions = np.delete(initial_conditions, j, 0)    # does the same process as before but for 
                    double_cycles = np.delete(double_cycles, j, 0)              # the comparing the negatives of the orbits
                j = j - 1                                                       # to the double cycles
            i = i + 1
        initial_conditions[initial_conditions == 1] = -1
        initial_conditions[initial_conditions == 2] = 0
        initial_conditions[initial_conditions == 3] = 1
        initial_conditions = initial_conditions*(-1)
        initial_conditions[initial_conditions == 1] = 3
        initial_conditions[initial_conditions == 0] = 2
        initial_conditions[initial_conditions == -1] = 1
    if check_rev == 1:
        initial_conditions = initial_conditions[...,::-1]
        i = 0
        while i < np.shape(initial_conditions)[0]:
            j = np.shape(initial_conditions)[0] - 1
            while j > i:
                if checkCyclic(initial_conditions[i],double_cycles[j]) == True:
                    initial_conditions = np.delete(initial_conditions, j, 0)
                    double_cycles = np.delete(double_cycles, j, 0)
                j = j - 1
            i = i + 1
    copy_of_reversed_initial = initial_conditions.copy()
    i = 0
    del_array = np.zeros(np.shape(initial_conditions)[0])
    while i < np.shape(initial_conditions)[0]:
        j = 1
        while j <= np.shape(initial_conditions)[1] - 1:
            rotate(copy_of_reversed_initial[i])
            if checkCyclic(copy_of_reversed_initial[i],initial_conditions[i]) == True:
                del_array[i] = 1
            j = j + 1
        i = i + 1
        
    initial_conditions = np.delete(initial_conditions, np.where(del_array == 1), 0)
    
    initial_conditions[initial_conditions == 1] = -1
    initial_conditions[initial_conditions == 2] = 0
    initial_conditions[initial_conditions == 3] = 1

    return initial_conditions

In [ ]:
def find_cycles(initial_conditions, n):
    """
    Returns all cycles of length n; numerical tolerance is not checked for each instance. 
    """
    # this creates all cycle combinations to use as initial conditions.
#     initial_conditions = list(itertools.product(*(shadow_states for i in range(n))))
    cycles = []
    # this runs the l-bfgs-b optimization algorithm for each initial condition
    with Parallel(n_jobs=-2) as parallel:
        cycles = parallel(delayed(minimize)(cost, phi, jac=jac, method='l-bfgs-b') for phi in initial_conditions)
    
    # scipy technically returns objects which describe the run time statistics; this simply returns
    # the final lattice states (may not be fully converged).
    
    # It returns them in an array of shape [N, 2]
    solutions = np.array([sol.x for sol in cycles])
    return solutions

In [ ]:
fig, ax = plt.subplots()
ax.scatter(initial_conditions[:, 0], initial_conditions[:, 1])
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.grid()

In [ ]:
from joblib import Parallel, delayed

In [ ]:
s = 2 + musqr

In [ ]:
def eqn(lattice_state):
    """
    lattice_state : np.ndarray
        Vector of field values at each lattice sit
    
    """
    # the equations of motion t+1 and t-1 terms are accomplished by the "roll" function = cyclic rotation
    cost_vector = (-1 * np.roll(lattice_state, -1) + (-1*(s-2)*lattice_state**3 +
                                               s*lattice_state) - np.roll(lattice_state, 1))
    # the l2 norm, giving us a scalar cost functional
    return cost_vector

def cost(lattice_state):
    return 0.5 * np.linalg.norm(eqn(lattice_state))**2

def jac(lattice_state):
    """
    Jacobian of the COST FUNCTION
    
    """
    F = eqn(lattice_state)
    JTF = -np.roll(F, 1) - np.roll(F, -1) + (-3 * (s - 2) * lattice_state**2 + s) * F  
    return JTF

In [ ]:
x = PhiK(n, k, musqr).generate_states(prime=False)
# x.states = x.prime_orbits(check_neg=check_neg, check_rev=check_rev)
initial_conditions = primeOrbits(x.states[::-1], check_neg=True, check_rev=False)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(initial_conditions[:, 0], initial_conditions[:, 1])
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.grid()

In [ ]:
converged_cycles = find_cycles(initial_conditions, n)
converged_cycles

In [ ]:
 converged_cycles.min(axis=1)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(converged_cycles[:, 0], converged_cycles[:, 1])
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.grid()

In [ ]:
n=3
x = PhiK(n, k, musqr).generate_states(prime=False)
# x.states = x.prime_orbits(check_neg=check_neg, check_rev=check_rev)



def vectorized_comparison(symbols, symbols_to_compare_against, offdiag=0):
    counts = np.char.count(symbols_to_compare_against.astype(str).reshape(-1, 1),
                           symbols.astype(str).reshape(1, -1))
#     return counts
    lower_idx = np.tril_indices(len(counts), offdiag)
    mask = np.zeros_like(counts)
    mask[lower_idx] = 1
    masked_counts = np.ma.masked_array(counts, mask=mask)
    return masked_counts


x = PhiK(n, k, musqr).generate_states(prime=False)
# x.states = x.prime_orbits(check_neg=check_neg, check_rev=check_rev)

initial_conditions = primeOrbits(x.states[::-1], check_neg=True, check_rev=True)

In [ ]:
prime_symbols = np.sort(np.array((2+initial_conditions).astype(str), dtype=object).sum(axis=1))
prime_idx = np.where([s in prime_symbols for s in symbols])[0]

converged_cycles = find_cycles(initial_conditions, n)
converged_cycles

# n=6

In [ ]:
for check_neg, check_rev in list(itertools.product([True, False], [True, False]))[::-1]:
    x = PhiK(n, k, musqr).generate_states(prime=False)
    x.states = x.states[::-1]
    states = x.states
    # states = initial_conditions
    if -1 in states:
        states = states + 2

    # symbol representation
    symbols = np.sort(np.array(states.astype(str), dtype=object).sum(axis=1))
    # double repeats
    doubles = symbols + symbols
    initial_conditions = x.prime_orbits(check_neg=check_neg, check_rev=check_rev)
    prime_symbols = np.sort(np.array((2+initial_conditions).astype(str), dtype=object).sum(axis=1))
    prime_idx = np.where([s in prime_symbols for s in symbols])[0]
    masked_counts = vectorized_comparison(symbols, doubles, offdiag=-1)
    masked_counts[prime_idx, :] *= 100
    fig,ax = plt.subplots(figsize=(10,10))
    im = ax.matshow(masked_counts)
    plt.colorbar(im)

In [ ]:
states = x.states
# states = initial_conditions
if -1 in states:
    states = states + 2

# symbol representation
symbols = np.sort(np.array(states.astype(str), dtype=object).sum(axis=1))
# double repeats
doubles = symbols + symbols

# the function which does the vectorized comparison
masked_counts = vectorized_comparison(symbols, doubles, offdiag=-1)

In [ ]:
symbols[0]

In [ ]:
symbols[3]

In [ ]:
symbols[::3]

In [ ]:

fig,ax = plt.subplots(figsize=(10,10))
im = ax.matshow(masked_counts)
plt.colorbar(im)

In [ ]:
masked_counts

In [ ]:
# The index positions of cycles which are prime (so far)
admissible_index = list(np.where(((masked_counts == 1).sum(axis=0) == 0))[0]) + [0]
# The comparison fails to capture cycles like '111', handle them explicitly
not_pure_cyclic = np.where(masked_counts.sum(axis=1) != 2)[0]
admissible_index = list(set(admissible_index).intersection(set(not_pure_cyclic)))
# keep only the admissible states
states = states[admissible_index]
doubles = doubles[admissible_index]

prime_symbols = np.sort(np.array((2+initial_conditions).astype(str), dtype=object).sum(axis=1))
masked_counts = vectorized_comparison(symbols, doubles, offdiag=-1)

In [ ]:
masked_counts.shape

In [ ]:

fig,ax = plt.subplots(figsize=(10,10))
im = ax.matshow(masked_counts)
plt.colorbar(im)

In [ ]:
# states = x.states
# if -1 in states:
#     states = states + 2

# # symbol representation
# symbols = np.sort(np.array(states.astype(str), dtype=object).sum(axis=1))
# # double repeats
# doubles = symbols + symbols

# # the function which does the vectorized comparison
# masked_counts = vectorized_comparison(symbols, doubles, offdiag=-1)

# masked_counts[prime_idx, :] *= 100

# fig,ax = plt.subplots(figsize=(10,10))
# im = ax.matshow(masked_counts)
# plt.colorbar(im)

# fig,ax = plt.subplots(figsize=(10,10))
# im = ax.matshow(masked_counts[prime_idx, :])
# plt.colorbar(im)

# fig,ax = plt.subplots(figsize=(10,10))
# im = ax.matshow(masked_counts)
# plt.colorbar(im)

symbols[9]

doubles

states = x.states
if -1 in states:
    states = states + 2

# symbol representation
symbols = np.sort(np.array(states.astype(str), dtype=object).sum(axis=1))
# double repeats
doubles = symbols + symbols

# the function which does the vectorized comparison
masked_counts = vectorized_comparison(symbols, doubles, offdiag=-1)

# The index positions of cycles which are prime (so far)
admissible_index = list(np.where(((masked_counts == 1).sum(axis=0) == 0))[0]) + [0]
# The comparison fails to capture cycles like '111', handle them explicitly
not_pure_cyclic = np.where(masked_counts.sum(axis=1) != 2)[0]
admissible_index = list(set(admissible_index).intersection(set(not_pure_cyclic)))
# keep only the admissible states
states = states[admissible_index]
doubles = doubles[admissible_index]